In [40]:
import pandas as pd


In [25]:
path_blast = 'data/rice/96.csv'
path_blast_filtered = 'data/rice/96.filtered.csv'


In [ ]:
path_blast = 'data/wheat/109136.csv'
path_blast_filtered = 'data/wheat/109136.filtered.csv'


In [41]:

path_blast = 'data/wheat/117188.csv'
path_blast_filtered = 'data/wheat/117188.filtered.csv'


In [43]:
#TEs
params = {'min_len':50,'max_len':False,'min_distance':5,'max_q':1.3,'min_q':0.9,'min_pident':96,'min_qcov':94}


In [27]:
#read blast output
df = pd.read_csv(path_blast, sep='\t', header=None)
df.columns = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs']
print('initial:',len(df.index))
initial = len(df.index)

initial: 98


In [28]:
#filter by length
if(params['min_len']):
    df = df[df.qlen > params['min_len']]
print('Min len: ' + str(len(df.index)))
min_length = str(len(df.index))

Min len: 98


In [29]:
if(params['max_len']):
    df = df[df.qlen < params['max_len']]
print('Max len: ' + str(len(df.index)))    
max_length = str(len(df.index))

Max len: 98


In [30]:
#filter by query / subject length treshold
df = df[((df.length / df.qlen) >= params['min_q'])]
print('min treshold:',len(df.index))
min_treshold = str(len(df.index))

min treshold: 51


In [31]:
df = df[((df.length / df.qlen) <= params['max_q'])]
print('max treshold:',len(df.index))
max_treshold = str(len(df.index))

max treshold: 51


In [32]:
#filter by pident
df = df[(df.pident >= params['min_pident'])]
print('Min_pident: ' + str(len(df.index)))
min_pident = str(len(df.index))

Min_pident: 51


In [33]:
#filter by qcov
df = df[(df.qcovs >= params['min_qcov'])]
print('Min qcov: ' + str(len(df.index)))
min_qcov = str(len(df.index))

Min qcov: 51


In [34]:
#order sstart and send
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1)
df = df.sort_values(by=['sseqid','sstart', 'send'])
df = df.reset_index(drop=True)
# sep by chr
dfs = {}
for seq in df.sseqid.unique():
    dfs[seq] = df[df.sseqid == seq]

In [35]:
df.head(10)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs
0,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr01,3,438,17513831,17514266,0,0,100.000,0.0,436,442,43270923,99
1,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr01,3,440,23332544,23332981,0,0,100.000,0.0,438,442,43270923,99
2,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr01,5,436,24779770,24780201,0,0,100.000,0.0,432,442,43270923,99
3,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr01,5,436,25261111,25261542,0,0,100.000,0.0,432,442,43270923,99
4,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr01,3,438,29931514,29931949,0,0,100.000,0.0,436,442,43270923,99
5,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr02,3,438,214434,214869,0,0,100.000,0.0,436,442,35937250,99
6,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr02,5,439,617948,618382,1,0,99.770,0.0,435,442,35937250,99
7,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr02,3,438,13161935,13162370,0,0,100.000,0.0,436,442,35937250,99
8,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr02,3,438,18534784,18535219,1,0,99.771,0.0,436,442,35937250,99
9,MITE_T_96|chr06|18136415|18136858|TA|25|F7_chr...,chr02,5,436,22549114,22549545,0,0,100.000,0.0,432,442,35937250,99


In [36]:
# filter overlapped 
rows = []
discard = []
total = len(df.index)
count = 0
curr = 0
for index, row in df.iterrows():
    count += 1
    curr_new = int(count * 100 * 1.0 / (total * 1.0))
    if curr_new != curr:
        curr = curr_new
        if curr_new % 1 == 0:
            print(curr_new)
    if index in discard:
        continue
    for k2, v2 in df.loc[index:,].iterrows():
        if abs(v2.sstart - row.sstart) > params['min_distance']:
            break
        if abs(v2.sstart - row.sstart) <= params['min_distance'] and abs(v2.send - row.send) <= params['min_distance']:
            discard.append(k2)
    rows.append(row)


1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100


In [37]:
df = pd.DataFrame(rows)
print('Non overlapped: ' + str(len(df.index)))
non_overlapped = str(len(df.index))


Non overlapped: 51


In [38]:
df.to_csv(path_blast_filtered, index=None, sep='\t')
path_blast_filtered

'data/rice/96.filtered.csv'

In [39]:
print('Initial: ' + str(initial))
print('Min len: ' + str(min_length))
print('Max len: ' + str(max_length))
print('Min treshold: ' + str(min_treshold))
print('Max treshold: ' + str(max_treshold))
print('Min pident: ' + str(min_pident))
print('Min qcov: ' + str(min_qcov))
print('Non overlapped: ' + str(non_overlapped))
print('Saved: ' + path_blast_filtered)

Initial: 98
Min len: 98
Max len: 98
Min treshold: 51
Max treshold: 51
Min pident: 51
Min qcov: 51
Non overlapped: 51
Saved: data/rice/96.filtered.csv
